# Projet maison n° 3

In [68]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [114]:
# names us
import glob
def df_names_us():
    #df = pd.DataFrame()

    names_us_files = [names_us_filename for names_us_filename in glob.glob("names_us/"+"*.txt")]
    
    lst_dfs = []
    for names_us_filename in names_us_files:
        df_tmp = pd.read_csv(names_us_filename, names=['name', 'gender', 'births'], header=None)
        df_tmp.insert(0, 'year', names_us_filename[12:16])
        lst_dfs.append(df_tmp)
        
    df = pd.concat(lst_dfs, ignore_index=True)
    return df

In [118]:
print (df_names_us())
df_names_us().info()

         year       name gender  births
0        1880       Mary      F    7065
1        1880       Anna      F    2604
2        1880       Emma      F    2003
3        1880  Elizabeth      F    1939
4        1880     Minnie      F    1746
...       ...        ...    ...     ...
2052776  2021     Zyeire      M       5
2052777  2021       Zyel      M       5
2052778  2021      Zyian      M       5
2052779  2021      Zylar      M       5
2052780  2021        Zyn      M       5

[2052781 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2052781 entries, 0 to 2052780
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    object
 1   name    object
 2   gender  object
 3   births  int64 
dtypes: int64(1), object(3)
memory usage: 62.6+ MB


In [296]:
import os
os.curdir()

Attribute references
********************

An attribute reference is a primary followed by a period and a name:

   attributeref ::= primary "." identifier

The primary must evaluate to an object of a type that supports
attribute references, which most objects do.  This object is then
asked to produce the attribute whose name is the identifier.  This
production can be customized by overriding the "__getattr__()" method.
If this attribute is not available, the exception "AttributeError" is
raised.  Otherwise, the type and value of the object produced is
determined by the object.  Multiple evaluations of the same attribute
reference may yield different objects.

Related help topics: getattr, hasattr, setattr, ATTRIBUTEMETHODS, FLOAT,
MODULES, OBJECTS



## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [116]:
# names fr
def df_names_fr1():

    df = pd.read_csv("names_fr/nat2021.csv",sep=';',dtype= str).drop(index = 0)
    df = df[['annais', 'preusuel', 'sexe', 'nombre']]
    df = df.rename(columns={"annais": "year", "preusuel": "name", "sexe":"gender", "nombre":"births"})   

    df['gender'] = df['gender'].apply(lambda x: 'M' if (x==1) else 'F')
        
    df = df[df['name'].apply(lambda x : len(str(x))) > 1]  
    
    df = df[df["name"] != "_PRENOMS_RARES"]
    df = df[df["year"] != "XXXX"] 
    df['name'] = df['name'].apply(lambda x : str(x).title()) 

    df = df.sort_values(by=["year", "gender", "births", "name"], ascending=[True, True, False, True])
    df.index = pd.RangeIndex(len(df))

    return df

In [236]:
# print(df_names_fr())
# df_names_us().info()

0          True
1          True
2          True
3          True
4          True
           ... 
2052776    True
2052777    True
2052778    True
2052779    True
2052780    True
Name: name, Length: 2052781, dtype: bool

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [178]:
def df_taux_change(devise_list):
    # PREPROCESSING.
    # Load dataset without unusefull header
    df = pd.read_csv('Webstat_Export.csv',sep =';', skiprows=[i for i in range(1,6)])
    # Remove empty rows
    condition = df["Dollar australien (AUD)"]!='-'
    df = df.loc[condition,:]
    # Remove akward column : Roupie Indienne (100 paise)
    df.drop("Roupie Indienne (100 paise)", axis=1, inplace=True)
    
    # COLUMN SELECTION.
    selected_columns = []
    columns = list(df.columns)
    
    for devise in devise_list:
        for column_name in columns:
            if devise in column_name:
                selected_columns.append(column_name)
    
    df = df[["Titre :"] + selected_columns]
    df.columns = ["Titre :"] + devise_list
    
    df = df.replace(',','.',regex=True)
    df[devise_list] = df[devise_list].astype('float')
    
    df = df.rename(columns={"Titre :": "Date"})
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y") 
    df = df.sort_values(by="Date", ascending=True)
    df = df.set_index("Date")

    return df

In [184]:
# print(df_taux_change(["CHF", "GBP", "USD"]))
# df_taux_change(["CHF", "GBP", "USD"]).info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6075 entries, 1999-01-04 to 2022-09-20
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CHF     6075 non-null   float64
 1   GBP     6075 non-null   float64
 2   USD     6075 non-null   float64
dtypes: float64(3)
memory usage: 189.8 KB


### Tests

In [181]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [182]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 7.164s

OK
